In [19]:
import os
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, KFold

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, Activation,UpSampling2D
from keras.layers.merge import add,concatenate
from keras.optimizers import Nadam, Adam, TFOptimizer, SGD, RMSprop, Nadam
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D,GlobalAveragePooling2D
from keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2

from keras.losses import binary_crossentropy

from keras.utils.vis_utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K

from yellowfin import YFOptimizer
from tqdm import tqdm

Using TensorFlow backend.


In [18]:
#Utils:

# import spectral as sp
# from skimage import io
from itertools import chain
# import spectral as sp
# from multiprocessing import Pool
# from skimage.transform import resize

# from concurrent.futures import ThreadPoolExecutor

import bcolz

In [4]:
path = '/media/gpk/d1c14824-369d-4058-9862-6319d625d0b8/PlanetAmazon'

In [5]:
#Reference: https://www.kaggle.com/ekami66/0-92837-on-private-lb-solution-with-keras
train_tiff_dir, test_tiff_dir, train_csv_file, test_mapping_file = data_helper.get_tiff_data_files_paths(path)
labels_df = pd.read_csv(train_csv_file)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [6]:
IMG_RESIZE=(128,128)
SPLIT_SIZE = 0.2
BATCH_SIZE = 128
NB_CLASSES = 17

In [7]:
labels_df = pd.read_csv(train_csv_file)
labels = sorted(set(chain.from_iterable([tags.split(" ") for tags in labels_df['tags'].values])))
labels_map = {l: i for i, l in enumerate(labels)}
y_map = {v: k for k, v in labels_map.items()}

In [8]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [78]:
x_train = load_array('input/x_train_90deg_128_128_3_wjpgs.bz/')
# x_train = load_array('input/x_train_128_128_7.bz')

y_train = load_array('input/y_train')

In [79]:
# x_test = load_array('input/x_test_64_64_5_wjpgs_ndwi_evi.bz')
x_test = load_array('input/x_test_90deg_128_128_3_jpgs.bz')
gc.collect()
# del x_train

0

In [ ]:
x_train.shape,y_train.shape,x_test.shape

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train,test_size=SPLIT_SIZE)


X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)

Y_train = Y_train.astype(np.float32)
Y_valid = Y_valid.astype(np.float32)


# del x_train

# Parallel GPU:

In [ ]:
def make_parallel(model, gpu_count):
    #https://medium.com/@kuza55/transparent-multi-gpu-training-on-tensorflow-with-keras-8b0016fd9012
    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([ shape[:1] // parts, shape[1:] ],axis=0)
        stride = tf.concat([ shape[:1] // parts, shape[1:]*0 ],axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    #Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                #Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx':i,'parts':gpu_count})(x)
                    inputs.append(slice_n)                

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                #Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(concatenate(outputs,axis=0))
#             merged.append(merge(outputs, mode='concat', concat_axis=0))
                
            
        return Model(input=model.inputs, output=merged)

# model = make_parallel(model,2)

# Metrics:

In [5]:
smooth = 1e-12
def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

# Models:

## Vgg style:

In [96]:
def _conv_block(x,nf,size,p=0,act=True):
    if act == True:
        x = Conv2D(nf,(size,size),padding='same',activation='relu')(x)
        x = Conv2D(nf,(size,size),activation='relu')(x)
        x = BatchNormalization()(x)
        
#         x = MaxPooling2D(pool_size=2)(x)
#         x = Dropout(p)(x)
        return Activation('relu')(x) if act else x

In [97]:
def vgg_block_model(inp,nf=512,p=0.2):
    
    x = _conv_block(inp,nf,3,p=p)
#     x = conv_block(inp,nf,3,(2,2))
#     x = AveragePooling2D(pool_size=2)(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(p)(x)
    
    x = _conv_block(x,nf,3,p=p)
#     x = AveragePooling2D(pool_size=2)(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(p)(x)
    
    x = _conv_block(x,nf,3,p=p)
#     x = AveragePooling2D(pool_size=2)(x)
    x = MaxPooling2D(pool_size=2)(x)
#     x = Dropout(p)(x)

    
    x = _conv_block(x,nf,3,p=p)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(p)(x)
    return x

In [99]:
def _get_vgg_model(arr,p=0.2):
    
    nf = 32
    inp=Input(arr.shape[1:])
    x = _conv_block(inp,nf,3,p=p)
#     x=conv_block(inp, nf, 3, (1,1))
#     growth = 2
    for i in range(3):
        x = _conv_block(x,nf,3,p=p)
        x = MaxPooling2D(pool_size=2)(x)
        x = Dropout(p)(x)
        nf = nf*2
        
#         nf = nf*growth
        
#     x= Convolution2D(17, 3, 3, activation='elu', border_mode='same')(x)
    x = Conv2D(17,(3,3),activation='relu',padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    outp = Activation('sigmoid')(x)

#     x = Flatten()(x)
#     x = Dense(512,activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(256,activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Dropout(0.8)(x)
#     outp = Dense(17,activation='sigmoid')(x)
    
    model = Model([inp],[outp])
    return inp,model

In [102]:
# inp,model = get_vgg_model(x_train)
inp,vgg_model = _get_vgg_model(x_train)

In [108]:
vgg_model.compile(optimizer=Nadam(),loss='binary_crossentropy',metrics=['accuracy',jaccard_coef])

In [104]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_130 (Conv2D)          (None, 126, 126, 32)      9248      
_________________________________________________________________
batch_normalization_65 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
activation_18 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_131 (Conv2D)          (None, 126, 126, 32)      9248      
_________________________________________________________________
conv2d_132 (Conv2D)          (None, 124, 124, 32)      9248      
__________

In [ ]:
opt = Nadam()
# opt = Adam()
# opt = SGD(0.1,0.9,nesterov=True)
# opt = TFOptimizer(YFOptimizer())
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['binary_accuracy',jaccard_coef,jaccard_coef_int])
K.set_value(model.optimizer.lr,0.01)

In [ ]:
#Reference: fast.ai

def bn(x):
    return BatchNormalization()(x)

def act(x):
    return Activation('relu')(x)

def conv(x,nf=32,wd=1e-4,p=0.2):
    x = Conv2D(nf, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(nf, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(p)(x)
    return x

def get_cov_model(arr,nblocks=4,wd=1e-4,p=0.2):
    
    inp = Input(shape=arr[1:])
    x = act(bn(x))
    
    for i in range(nblocks):
        x = conv(x,nf=32*i,wd=wd,p=p)(x)
    
    x = Flatten()(x)
    x = Dense(512,activation='relu',kernel_regularizer=l2(wd))
    x = bn(x)
    x = Dense(17,activation='relu',kernel_regularizer=l2(wd))
    


In [ ]:
# model = make_parallel(model,2) 
model.fit(X_train,Y_train,batch_size=256,epochs=3,validation_data=[X_valid,Y_valid],callbacks=callback_list)

In [ ]:
predictions = model.predict(x_test)

# Callbacks:

## Loss History

In [14]:
class LossHistory(Callback):
    #Reference: 
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

In [15]:
def sch(epoch):
    if epoch%20 == 0:
        lr = 1e-3
    if epoch%40 == 0:
        lr = 1e-4
    return lr

reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,mode='auto',verbose=2,min_lr=1e-8)
# earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# snapshot = ModelCheckpoint(filepath="snapshots/densenet_weights._{epoch:04d}-val_loss-{val_loss:.4f}.h5",monitor="val_loss",
#                            save_best_only=False)
# tqdm = TQDMNotebookCallback()

filepath="snapshots/unet_m_dummy_128_128_3.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
# lr_sch = LearningRateScheduler(sch)
callback_list = [ reduce_lr,checkpoint]

# Unet:

In [6]:
#Reference: https://www.kaggle.com/drn01z3/end-to-end-baseline-with-u-net-keras

def bn(x):
    return BatchNormalization()(x)

In [7]:
def pool2d(x):
    return MaxPooling2D(pool_size=(2,2))(x)

In [8]:
def act(x):
    return Activation('relu')(x)

In [9]:
def up2d(x):
    return UpSampling2D(size=(2,2))(x)

In [10]:
def drp(x,p):
    return Dropout(p)(x) if p else x

In [11]:
def conv_block(x,nf,size,act=False):
    x = Conv2D(nf,(size,size),padding='same',activation='relu')(x)
    x = Conv2D(nf,(size,size),padding='same', activation='relu')(x)
    return Activation(x) if act else x


def unet_conv_block(x,nf,growth_step,positive_growth=True):
    if positive_growth is True: nf = nf+growth_step
    else: nf = nf-growth_step
    return conv_block(x,nf,3)

In [12]:
def unet_conv_block(x,nf,growth_step,positive_growth=True):
    if positive_growth is True: nf = nf*growth_step
    else: nf = nf//growth_step
    return conv_block(x,nf,3)

In [13]:
def get_unet(arr,nf=32,growth_step=32):
    inp = Input(shape=arr.shape[1:])
    nf = 32
#     growth_rate = 2
    growth_step = 1
    
    conv1 = conv_block(bn(inp),nf,3)
    pool1 = bn(pool2d(conv1))
    
    conv2 = unet_conv_block(pool1,nf*2,growth_step,positive_growth=True)
    pool2 = bn(pool2d(conv2))

    conv3 = unet_conv_block(pool2,nf*2,growth_step,positive_growth=True)
    pool3 = bn(pool2d(conv3))
    
    conv4 = unet_conv_block(pool3,nf*4,growth_step,positive_growth=True)
    pool4 = bn(pool2d(conv4))
    
    conv5 = unet_conv_block(pool4,nf*8,growth_step,positive_growth=True)
    pool5 = bn(pool2d(conv5))
    
    #Lower most convs in the U.
    conv6 = act(Conv2D(nf*16,(3,3),name='bottom_0',padding='same')(pool5))
    conv6 = act(Conv2D(nf*16,(3,3),name='bottom_1',padding='same')(conv6))
#     conv6  = conv_block(bn(pool5),nf*16,3)
    
    up6 = up2d(conv6)
    up6 = bn(concatenate([up6,conv5],axis=-1))
    
    conv7  = unet_conv_block(up6,nf*8,growth_step,positive_growth=False)
    up7 = up2d(conv7)
    up7 = bn(concatenate([up7,conv4],axis=-1))
    
    conv8  = unet_conv_block(up7,nf*4,growth_step,positive_growth=False)
    up8 = up2d(conv8)
    up8 = bn(concatenate([up8,conv3],axis=-1))
    
    conv9  = unet_conv_block(up8,nf*2,growth_step,positive_growth=False)
    up9 = up2d(conv9)
    up9 = bn(concatenate([up9,conv2],axis=-1))
    
    #Changed this part of the Network compared to default implementation:
    
    conv10  = conv_block(up9,nf,3,act=False)
    pool10 = pool2d(conv10)
    #FCN head , added dropout
#     fcn_op = drp(Conv2D(128,(3, 3), padding='same', activation='relu')(conv10), 0.5)
    fcn_op = drp(Conv2D(32,(3, 3), padding='same', activation='relu')(pool10),0.1)
    fcn_op = Conv2D(17,(3, 3), padding='same', activation='sigmoid')(fcn_op)
    
#     outp = GlobalAveragePooling2D()(conv10)
    outp = GlobalAveragePooling2D()(fcn_op)
    return inp,outp

    

In [80]:
inp,outp = get_unet(x_train)

In [81]:
unet_m = Model(inp,outp)

In [82]:
unet_m.summary()
plot_model(unet_m,'modified_unet.png')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
batch_normalization_51 (BatchNor (None, 128, 128, 3)   12          input_6[0][0]                    
____________________________________________________________________________________________________
conv2d_100 (Conv2D)              (None, 128, 128, 32)  896         batch_normalization_51[0][0]     
____________________________________________________________________________________________________
conv2d_101 (Conv2D)              (None, 128, 128, 32)  9248        conv2d_100[0][0]                 
___________________________________________________________________________________________

In [87]:
opt = Nadam()
# opt = TFOptimizer(YFOptimizer())
unet_m.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy',jaccard_coef])
# K.set_value(unet_m.optimizer.learn_rate,1)

In [14]:
unet_m.fit(X_train,Y_train,validation_data=[X_valid,Y_valid],batch_size=128,epochs=2)

In [ ]:
unet_m_predictions = unet_m.predict(x_test)

# Cross-Validation:

In [15]:
def evaluate_model(model,X_valid,Y_valid,thres=0.18):
        p_valid = model.predict(X_valid)
        
        thresholded_preds = np.array(p_valid) > thres
        
        return fbeta_score(Y_valid, thresholded_preds, beta=2, average='samples')#,thresholded_preds

In [16]:
def run_cv(model,X, y, nfolds=2,lr=1e-3, epochs=1, batch_size=128, split_size=0.1,train_callbacks=()):
        '''Input compiled model and data(X,y)'''
        
        sum_scores = 0
        num_fold = 0
        y_val_preds_dict = {}
        
        kf = KFold(n_splits=nfolds)

        for train_index, test_index in kf.split(X,y):
            gc.collect()
            X_train = X[train_index]
            Y_train = y[train_index]
            X_valid = X[test_index]
            Y_valid = y[test_index]
            
            gc.collect()
            
            num_fold += 1
            print('==Start Fold number {} from {}'.format(num_fold, nfolds))
            print('===Split train: ', len(X_train), len(Y_train))
            print('===Split valid: ', len(X_valid), len(Y_valid))

            history = LossHistory()  
            earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')
            callbacks = [history,earlyStopping]
            
            K.set_value(model.optimizer.lr,lr)
            model.fit(X_train, Y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(X_valid, Y_valid),
                            callbacks=[history,earlyStopping,*callbacks])
            gc.collect()
            fbeta = evaluate_model(model,X_valid,Y_valid,thres=0.2)

            
            print('fbeta score: ', fbeta)                     
            sum_scores += fbeta*len(test_index)
            
        score = sum_scores /  len(x_train)
        print("fbeta val score on whole train set: ", score)

        return [history.train_losses,history.val_losses, score]

In [88]:
unet_m.load_weights('snapshots/CV_10_fold_unet_m_128_128_3_jpgs.h5')

In [48]:
# unet.fit(X_train, nb_epoch=100)
unet_m.save_weights('snapshots/CV_10_fold_unet_m_128_128_3_jpgs.h5')

In [17]:
#Debugging:

In [ ]:

inp = Input(x_train.shape[1:])
nf = 32
growth_step = 32
    
x = bn(inp)
x = act(Conv2D(nf,(3,3),padding='same')(x))
x = act(Conv2D(nf,(3,3),padding='same')(x))
x = pool2d(x)
pool1 = bn(x)
conv2 = unet_conv_block(pool1,nf,growth_step,positive_growth=True)
# pool2 = pool2d(conv2)

In [ ]:
type(conv2)

In [ ]:
graph = act(Conv2D(32,(3,3),padding='same')(bn(Input(x_train.shape[1:]))))

In [ ]:
type(graph)

In [ ]:
y_map

In [ ]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']


In [ ]:
climate_idx = np.array([labels_map[label] for label in weather_labels])
climate_idx

In [ ]:
y_train[:,idx].shape

In [ ]:
def drop_columns(y,drop_idx):
    return y[:,idx]

In [ ]:
Y_cli_train = drop_columns(Y_train,climate_idx)
Y_cli_valid = drop_columns(Y_valid,climate_idx)

In [109]:
train_losses, val_losses = [], []
epochs_arr = [10,5,5]
learn_rates = [0.001,0.0001,0.00001]#, 0.00001]


for learn_rate, epochs in zip(learn_rates, epochs_arr):
    tmp_train_losses, tmp_val_losses, score = run_cv(vgg_model,x_train, y_train,
                                                           nfolds=10,
                                                           lr=learn_rate, epochs=epochs, 
                                                           batch_size=BATCH_SIZE,
                                                           train_callbacks=callback_list)
    train_losses.append(tmp_train_losses)
    val_losses.append(tmp_val_losses)

==Start Fold number 1 from 10
===Split train:  36431 36431
===Split valid:  4048 4048
Train on 36431 samples, validate on 4048 samples
Epoch 1/10
36431/36431 [==============================] - 110s - loss: 0.6451 - acc: 0.7423 - jaccard_coef: 0.2162 - val_loss: 0.6919 - val_acc: 0.8182 - val_jaccard_coef: 0.1455
Epoch 2/10
36431/36431 [==============================] - 110s - loss: 0.6351 - acc: 0.8617 - jaccard_coef: 0.2272 - val_loss: 0.6463 - val_acc: 0.8777 - val_jaccard_coef: 0.2056
Epoch 3/10
36431/36431 [==============================] - 110s - loss: 0.6325 - acc: 0.8910 - jaccard_coef: 0.2300 - val_loss: 0.6353 - val_acc: 0.9187 - val_jaccard_coef: 0.2246
Epoch 4/10
36431/36431 [==============================] - 111s - loss: 0.6307 - acc: 0.9000 - jaccard_coef: 0.2320 - val_loss: 0.6553 - val_acc: 0.9060 - val_jaccard_coef: 0.2170
Epoch 5/10
36431/36431 [==============================] - 110s - loss: 0.6294 - acc: 0.9088 - jaccard_coef: 0.2333 - val_loss: 0.6372 - val_acc: 0.90

36431/36431 [==============================] - 111s - loss: 0.6159 - acc: 0.9448 - jaccard_coef: 0.2481 - val_loss: 0.6351 - val_acc: 0.9162 - val_jaccard_coef: 0.2273
Epoch 2/10
36431/36431 [==============================] - 112s - loss: 0.6158 - acc: 0.9454 - jaccard_coef: 0.2482 - val_loss: 0.6290 - val_acc: 0.9303 - val_jaccard_coef: 0.2367
Epoch 3/10
36431/36431 [==============================] - 111s - loss: 0.6153 - acc: 0.9451 - jaccard_coef: 0.2486 - val_loss: 0.6201 - val_acc: 0.9426 - val_jaccard_coef: 0.2414
Epoch 4/10
36431/36431 [==============================] - 111s - loss: 0.6149 - acc: 0.9457 - jaccard_coef: 0.2488 - val_loss: 0.6467 - val_acc: 0.9242 - val_jaccard_coef: 0.2303
Epoch 5/10
36431/36431 [==============================] - 111s - loss: 0.6149 - acc: 0.9461 - jaccard_coef: 0.2488 - val_loss: 0.6258 - val_acc: 0.9352 - val_jaccard_coef: 0.2427
fbeta score:  0.485202774431
==Start Fold number 10 from 10
===Split train:  36432 36432
===Split valid:  4047 4047


KeyboardInterrupt: 

In [71]:
unet_m.fit(x_train,y_train,batch_size=128,epochs=1)

Epoch 1/1
40479/40479 [==============================] - 112s - loss: 0.1816 - acc: 0.9270 - jaccard_coef: 0.5258   


In [ ]:
#Checking predictions :

#Looking at borderline cases(low max and/or high low values)
thres = 0.4
# high_min = {i:min(pred) for i,pred in enumerate(predictions) if min(pred)>=1e-3}
low_max = {i:max(pred) for i,pred in enumerate(predictions) if max(pred)<=0.45}

In [ ]:
high_min = {i:min(pred) for i,pred in enumerate(predictions) if max(pred)<=0.45}

In [ ]:
print(len(high_min))
high_min

In [ ]:
#Checking intersections:

len(set(low_max.keys()).intersection(set(high_min.keys())))

In [ ]:
print(len(low_max))
low_max

In [ ]:
{i:pred for i,pred in enumerate(dense_unet_predictions) if max(pred)<=thres}

In [ ]:
idx = np.array(list(low_max.keys())); print(idx)

In [ ]:
#Edge cases dict:
thres = 0.2
edge_cases_dict = {idx[i]:[y_map[j] for j,value in enumerate(pred) if value>thres] for i,pred in 
                   enumerate(predictions[idx,:])}

In [ ]:
edge_cases_dict

# Generate preds

In [91]:
thres = 0.2
y_pred_list = [' '.join([y_map[j] for j,value in enumerate(pred) if value>thres]) for i,pred in 
               enumerate(predictions_90)]

In [92]:
y_pred_list[:10]

['clear primary',
 'clear primary',
 'agriculture clear primary road',
 'clear primary water',
 'clear primary road',
 'haze primary',
 'clear primary',
 'agriculture partly_cloudy primary water',
 'clear primary',
 'partly_cloudy primary']

In [93]:
pred_dict = {op[0].split('.')[0]:op[1] for i,op in enumerate(zip(os.listdir(test_tiff_dir),y_pred_list))}

In [94]:
out_df = pd.DataFrame(list(pred_dict.items()),columns=['image_name', 'tags'])

In [95]:
out_df.to_csv('u_net_m_preds_da_1.csv', index=False)